In [28]:
from dotenv import dotenv_values
from SPARQLWrapper import SPARQLWrapper, POST, JSON
import requests

### Selección de BOE

In [29]:
boe_id = "BOE-A-2023-20397"

In [30]:
env_config = dotenv_values("../.env")

In [31]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

In [32]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

In [33]:
def get_titulos(boe_ids):
    sparql.setQuery(PREFIXES + f"""
    SELECT ?entradaBOE ?titulo
    WHERE {{
    ?entradaBOE rdf:type :EntradaBOE .
    ?entradaBOE :identificador "{boe_id}" .
    ?entradaBOE :titulo ?titulo .
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]

### Obtención anteriores y posteriores

In [34]:
def get_referenced_boes(boe_id):
    query = f"""
                PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
                PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

                SELECT ?entradaBOE ?boe_anterior ?boe_posterior
                WHERE {{
                    ?entradaBOE rdf:type :EntradaBOE .
                    ?entradaBOE :identificador "{boe_id}" .

                    OPTIONAL {{
                    ?entradaBOE :publica ?boe_anterior .
                    FILTER EXISTS {{ ?entradaBOE :publica ?boe_anterior }}
                    }}

                    OPTIONAL {{
                    ?entradaBOE :sePublica ?boe_posterior .
                    FILTER EXISTS {{ ?entradaBOE :sePublica ?boe_posterior }}
                    }}
                }}
            """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addParameter('Content-Type', 'application/sparql-query')
    results = sparql.query().convert()

    results = sparql.query().convert()
    anteriores = []
    posteriores = []
    for result in results["results"]["bindings"]:
        entradaBOE = result["entradaBOE"]["value"].split("/")[-1]
        print
        try:
            anteriores.append(result["boe_anterior"]["value"].split("/")[-1])
        except:
            pass
        try:
            posteriores.append(result["boe_posterior"]["value"].split("/")[-1])
        except:
            pass
    
    return {'anteriores': anteriores, 'posteriores': posteriores}

In [35]:
refs = get_referenced_boes(boe_id)
print(f"Entrada seleccionada del BOE: {boe_id}")

print("\nBOEs anteriores:")
for ref in refs['anteriores']:
    print(f"\t*{ref}: {get_titulos(ref)['titulo']['value']}")

print("\nBOEs posteriores:")
for ref in refs['posteriores']:
    print(f"\t*{ref}: {get_titulos(ref)['titulo']['value']}")

Entrada seleccionada del BOE: BOE-A-2023-20397

BOEs anteriores:
	*BOE-A-2022-4645: Resolución de 21 de septiembre de 2023, de las Presidencias del Congreso de los Diputados y del Senado, por la que se nombra personal funcionario del Cuerpo de Archiveros-Bibliotecarios de las Cortes Generales.

BOEs posteriores:
